<a href="https://colab.research.google.com/github/azad712/Image-classification-with-4-classes-/blob/main/mask_and_glasses_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import shutil
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = {'person': 0, 'person_wg': 1, 'person_wm':2, 'person_wgm':3}
        self.data = self.load_data()

    def load_data(self):
        data = []
        for filename in os.listdir(self.root_dir):
            if filename.endswith(".jpg"):
                label_str = filename.split('.')[0]
                label = self.classes.get(label_str, -1)
                if label != -1:
                    data.append({'image_path': os.path.join(self.root_dir, filename), 'label': label})
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]['image_path']
        label = self.data[idx]['label']
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Define data transformations
# Define the transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),        # Randomly flip the image horizontally
    transforms.RandomVerticalFlip(),          # Randomly flip the image vertically
    transforms.RandomRotation(20),            # Randomly rotate the image by 20 degrees
    transforms.RandomResizedCrop(224),        # Randomly crop and resize the image to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change the brightness, contrast, saturation, and hue
    transforms.ToTensor(),                    # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    # Normalize the image with mean and standard deviation
])

transform_test = transforms.Compose([
    transforms.Resize(256),                   # Resize the image to 256x256
    transforms.CenterCrop(224),               # Crop the center of the image to 224x224
    transforms.ToTensor(),                    # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    # Normalize the image with mean and standard deviation
])


# Specify the path to your dataset directory
dataset_path = "/content/drive/My Drive/mandg/train"

# Create an ImageFolder dataset
train_dataset = ImageFolder(root=dataset_path, transform=transform_train)
# Create a DataLoader to handle batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet model
resnet_model = models.resnet50(pretrained=True)

# Modify the fully connected layer
#num_classes = len(set(train_dataset.classes.values()))  # Get the number of classes dynamically
num_classes = 4
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9,  weight_decay=1e-5)
#print("done training")




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 141MB/s]


In [6]:
class ResNet50WithDropout(nn.Module):
    def __init__(self, original_model, p=0.5):
        super(ResNet50WithDropout, self).__init__()
        self.original_model = original_model
        self.dropout = nn.Dropout(p)

        # Optionally, you can add dropout layers to specific layers in the original model
        # For example, adding dropout after the average pooling layer:
        self.features = nn.Sequential(*list(original_model.children())[:-2])  # Everything except the last two layers
        self.avgpool = original_model.avgpool
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p),
            original_model.fc
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x

# Instantiate the modified model with dropout
model_with_dropout = ResNet50WithDropout(resnet_model, p=0.5)

In [ ]:
l1_lambda = 1e-5

for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_with_dropout(inputs)
        loss = criterion(outputs, labels)
        l1_norm = sum(param.abs().sum() for param in model_with_dropout.parameters())
        loss = loss + l1_lambda * l1_norm
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print("epoch number completed is ", epoch)

print('Finished Training')

epoch number completed is  0
epoch number completed is  1
epoch number completed is  2
epoch number completed is  3
epoch number completed is  4


In [ ]:
# Define the file path
directory = r'/content/drive/My Drive/mandg/'
file_path = os.path.join(directory, 'model.pth')

# Ensure the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the model's state dictionary
torch.save(model_with_dropout.state_dict(), file_path)

print(f'Model saved to {file_path}')


Model saved to /content/drive/My Drive/mandg/model.pth


In [9]:
# Load the state dictionary
model_with_dropout.load_state_dict(torch.load('/content/drive/My Drive/mandg/model.pth'))

# Set the model to evaluation mode (optional, but recommended if you're not training)
model_with_dropout.eval()


ResNet50WithDropout(
  (original_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seq

In [ ]:
testset_path =  "/content/drive/My Drive/mandg/test"
# Create an ImageFolder dataset
test_dataset = ImageFolder(root=testset_path, transform=transform_test)
# Create a DataLoader to handle batching and shuffling
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
model_with_dropout.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = resnet_model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        #print("labels", labels)
        #print("predicted", predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
